<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EC%9E%A5%EA%B8%B0/koalpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## koalpaca 사용해보기
- 참고:https://discuss.pytorch.kr/t/koalpaca/1342/2

#### 원본 사용 불가

In [ ]:
!git clone https://github.com/Beomi/KoAlpaca.git

In [ ]:
%cd KoAlpaca

[Errno 2] No such file or directory: 'KoAlpaca'
/content


In [ ]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
!python download.py

python3: can't open file '/content/download.py': [Errno 2] No such file or directory


### 새로운 방법 - 로드 불가
- https://domdom.tistory.com/622

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install sentencepiece
!pip install bitsandbytes
!pip install datasets
!pip install loralib

  Resolved https://github.com/huggingface/transformers to commit 2518e368105a78f6cdc50ded6971712f2c1e7ac4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
### 허깅페이스에서 코알파카 받아서 텍스트 나오게 해보기
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B")
model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B")

text = "안녕하세요, 저는 [MASK]를 좋아합니다."
input_ids = tokenizer.encode(text, return_tensors="pt")
mask_idx = input_ids[0].tolist().index(tokenizer.mask_token_id)

output = model(input_ids)
token_id = output.logits[0, mask_idx].argmax(dim=-1).item()
predicted_token = tokenizer.decode([token_id])

print(f"예측된 토큰: {predicted_token}")

In [ ]:
### 허깅페이스에서 코알파카 받아서 텍스트 나오게 해보기
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B")
model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-5.8B")
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
    ),
}

def gen(prompt, user_input=None, max_new_tokens=128, temperature=0.5):
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)

    input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
    gen_tokens = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)

    return gen_text.replace(x, '')

# Chat
prompt = "오늘 점심 메뉴를 추천해주세요."
generated_text = gen(prompt)
print(generated_text)

### 블로그 따라하기
- https://zayunsna.github.io/blog/2023-08-01-koalpaka/

In [ ]:
!pip install transformers
!pip install peft
!pip install datasets

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from datasets import load_dataset

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

data = load_dataset("beomi/KoAlpaca-v1.1a")
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

model_id = "./KoAlpaca_v1.1_save"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print("check Pretrained model's paramsters : ", print_trainable_parameters(model))

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # max_steps=50, ## 초소량만 학습: 50 step만 학습. 약 4분정도 걸립니다.
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

PackageNotFoundError: ignored

### 고려대학교 LLM - RAM 터짐
- https://github.com/nlpai-lab/KULLM

In [ ]:
!git clone https://github.com/nlpai-lab/KULLM.git

In [ ]:
!pip install -U torch transformers tokenizers accelerate

In [ ]:
%cd /content/KULLM

/content/KULLM


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from utils.prompter import Prompter

MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

pipe = pipeline("text-generation", model=model, tokenizer=MODEL, device=0)

prompter = Prompter("kullm")


def infer(instruction="", input_text=""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = pipe(prompt, max_length=512, temperature=0.2, num_beams=5, eos_token_id=2)
    s = output[0]["generated_text"]
    result = prompter.get_response(s)

    return result


result = infer(input_text="고려대학교에 대해서 알려줘")
print(result)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset

ds = load_dataset("nlpai-lab/kullm-v2", split="train")
ds
# DatasetDict({
#     train: Dataset({
#         features: ['id', 'instruction', 'input', 'output'],
#         num_rows: 152630
#     })
# })

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})

### .

In [ ]:
!pip install transformers

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-1.3b")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/polyglot-ko-1.3b")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### 학습 데이터 생성
- nlpai-lab/kullm-v2 데이터 세트 참고

In [1]:
import pandas as pd

In [12]:
to1 = pd.read_excel('/content/drive/MyDrive/1193886_30000.xls')
to2 = pd.read_excel('/content/drive/MyDrive/1193886_51960.xls')
total = pd.concat([to1,to2])
len(total)

74935

In [15]:
total.keys()

Index(['표제어', '동형어 번호', '구분', '품사', '고유어 여부', '원어', '발음', '활용', '파생어', '가봐라',
       '어휘 등급', '의미 범주', '주제 및 상황 범주', '전체 참고', '검색용 이형태', '관련어', '의미 참고',
       '다중 매체 정보', '문형', '문형 참고', '뜻풀이', '용례', '몽골어 대역어', '몽골어 대역어 뜻풀이',
       '베트남어 대역어', '베트남어 대역어 뜻풀이', '타이어 대역어', '타이어 대역어 뜻풀이', '인도네시아어 대역어',
       '인도네시아어 대역어 뜻풀이', '러시아어 대역어', '러시아어 대역어 뜻풀이', '영어 대역어', '영어 대역어 뜻풀이',
       '일본어 대역어', '일본어 대역어 뜻풀이', '프랑스어 대역어', '프랑스어 대역어 뜻풀이', '스페인어 대역어',
       '스페인어 대역어 뜻풀이', '아랍어 대역어', '아랍어 대역어 뜻풀이', '중국어 대역어', '중국어 대역어 뜻풀이',
       '부표제어', '부표제어 전체 참고', '부표제어 관련어', '부표제어 의미 참고', '부표제어 문형', '부표제어 문형 참고',
       '부표제어 뜻풀이', '부표제어 용례', '몽골어 부표제어 대역어', '몽골어 부표제어 대역어 뜻풀이',
       '베트남어 부표제어 대역어', '베트남어 부표제어 대역어 뜻풀이', '타이어 부표제어 대역어',
       '타이어 부표제어 대역어 뜻풀이', '인도네시아어 부표제어 대역어', '인도네시아어 부표제어 대역어 뜻풀이',
       '러시아어 부표제어 대역어', '러시아어 부표제어 대역어 뜻풀이', '영어 부표제어 대역어', '영어 부표제어 대역어 뜻풀이',
       '일본어 부표제어 대역어', '일본어 부표제어 대역어 뜻풀이', '프랑스어 부표제어 대역어',
       '프랑스어 부표제어 대역어 뜻풀이', '스페인어 부표제어 대역어', '스페인어 부표제어 대역어

In [14]:
# 결측치 확인
total.isnull().sum()

표제어                      0
동형어 번호                   0
구분                       0
품사                       0
고유어 여부                3194
                     ...  
스페인어 부표제어 대역어 뜻풀이    71742
아랍어 부표제어 대역어         71743
아랍어 부표제어 대역어 뜻풀이     71743
중국어 부표제어 대역어         71741
중국어 부표제어 대역어 뜻풀이     71741
Length: 74, dtype: int64

In [18]:
t_data = total[['표제어', '동형어 번호', '구분', '품사', '고유어 여부','어휘 등급','뜻풀이', '용례']]
t_data

,표제어,동형어 번호,구분,품사,고유어 여부,어휘 등급,뜻풀이,용례
0,가,1,단어,명사,고유어,고급,어떤 장소나 물건의 둘레나 끝부분.,<구> 가를 꾸미다.\n<구> 가를 장식하다.\n<구> 가에 걸치다.\n<구> 가에...
1,가,1,단어,명사,고유어,고급,‘주변’의 뜻을 나타내는 말.,<구> 강가.\n<구> 길가.\n<구> 냇가.\n<구> 문가.\n<구> 시냇가.\n...
2,-가,1,단어,접사,한자어,없음,‘그 성을 가진 사람’의 뜻을 더하는 접미사.,<구> 김가\n<구> 박가\n<구> 이가\n\n
3,가-,1,단어,접사,한자어,없음,‘가짜의’ 또는 ‘임시의’의 뜻을 더하는 접두사.,<구> 가건물\n<구> 가계약\n<구> 가등기\n<구> 가매장\n<구> 가석방\n\n
4,가,2,단어,명사,한자어,없음,회의 등에서 어떤 안건에 대해 표결을 할 때 찬성한다는 표시.,<구> 가와 부.\n<구> 가를 밝히다.\n<문장> 곧 표결을 시작하오니 가인지 부...
...,...,...,...,...,...,...,...,...
31244,ㅟ,0,단어,명사,고유어,없음,한글 자모 ‘ㅜ’와 ‘ㅣ’를 모아 쓴 글자. 이름은 ‘위’이고 중성으로 쓴다.,NaN
31245,ㅠ,0,단어,명사,고유어,없음,한글 자모의 스물두째 글자. 이름은 ‘유’이고 중성으로 쓴다.,NaN
31246,ㅡ,0,단어,명사,고유어,없음,한글 자모의 스물셋째 글자. 이름은 ‘으’이고 중성으로 쓴다.,NaN
31247,ㅢ,0,단어,명사,고유어,없음,한글 자모 ‘ㅡ’와 ‘ㅣ’를 모아 쓴 글자. 이름은 ‘의’이고 중성으로 쓴다.,NaN


In [17]:
t_data.isnull().sum()

표제어               0
동형어 번호            0
구분                0
품사                0
고유어 여부         3194
원어            32821
발음            11448
활용            56313
파생어           61654
가봐라           61554
어휘 등급          3194
의미 범주         63474
주제 및 상황 범주    64969
전체 참고         72264
검색용 이형태       72771
관련어           52728
의미 참고         70608
다중 매체 정보      70977
문형            51860
문형 참고         74935
뜻풀이            3194
용례             5313
dtype: int64

In [39]:
train_data = t_data.dropna(subset=['용례'])
train_data.isnull().sum()

표제어       0
동형어 번호    0
구분        0
품사        0
고유어 여부    0
어휘 등급     0
뜻풀이       0
용례        0
dtype: int64

In [40]:
train_data['품사'].value_counts()

명사        35274
동사        18675
부사         4849
형용사        4523
품사 없음      2468
관형사         976
어미          754
접사          686
의존 명사       416
조사          359
감탄사         262
대명사         209
수사           82
보조 동사        65
보조 형용사       24
Name: 품사, dtype: int64

In [41]:
train_data.keys()

Index(['표제어', '동형어 번호', '구분', '품사', '고유어 여부', '어휘 등급', '뜻풀이', '용례'], dtype='object')

In [42]:
train = train_data[train_data['품사'].isin(['명사', '동사', '부사','형용사', '관형사', '대명사', '수사'])]
train

,표제어,동형어 번호,구분,품사,고유어 여부,어휘 등급,뜻풀이,용례
0,가,1,단어,명사,고유어,고급,어떤 장소나 물건의 둘레나 끝부분.,<구> 가를 꾸미다.\n<구> 가를 장식하다.\n<구> 가에 걸치다.\n<구> 가에...
1,가,1,단어,명사,고유어,고급,‘주변’의 뜻을 나타내는 말.,<구> 강가.\n<구> 길가.\n<구> 냇가.\n<구> 문가.\n<구> 시냇가.\n...
4,가,2,단어,명사,한자어,없음,회의 등에서 어떤 안건에 대해 표결을 할 때 찬성한다는 표시.,<구> 가와 부.\n<구> 가를 밝히다.\n<문장> 곧 표결을 시작하오니 가인지 부...
5,가,2,단어,명사,한자어,없음,"성적이나 등급을 나타내는 ‘수’, ‘우’, ‘미’, ‘양’, ‘가’의 다섯 단계 중...",<구> 가가 나오다.\n<구> 가를 받다.\n<문장> 운동을 못 하는 지수는 체육 ...
6,가,2,단어,명사,한자어,없음,어떤 행동이 가능하거나 허용됨.,<구> 관람 가.\n<구> 무이자 가.\n<구> 선택 가.\n<구> 입장 가.\n<...
...,...,...,...,...,...,...,...,...
30900,힘주다,0,단어,동사,고유어,고급,말에 강조하는 뜻을 갖게 하다.,<구> 힘주어 강조하다.\n<구> 힘주어 대답하다.\n<구> 힘주어 말하다.\n<구...
30901,힘줄,0,단어,명사,고유어,없음,근육을 이루는 희고 질긴 살의 줄.,<구> 굵은 힘줄.\n<구> 힘줄이 나타나다.\n<구> 힘줄이 보이다.\n<구> 힘...
30903,힘차다,0,단어,형용사,고유어,중급,힘이 있고 씩씩하다.,<구> 힘찬 날갯짓.\n<구> 힘찬 목소리.\n<구> 힘찬 움직임.\n<구> 힘차게...
30904,힙합,0,단어,명사,외래어,없음,1980년대에 미국의 흑인 젊은이들 사이에서부터 생긴 새로운 감각의 음악이나 춤.,<구> 힙합 가수.\n<구> 힙합 문화.\n<구> 힙합 장르.\n<구> 힙합을 듣다...


In [46]:
## 예문, 뜻 만들 수 있는지 확인
a = train[(train['표제어']=='가')]
뜻 = '\n'.join(list(a['뜻풀이']))
print(뜻)
예문 = list(a['용례'])
print(예문)

어떤 장소나 물건의 둘레나 끝부분.
‘주변’의 뜻을 나타내는 말.
회의 등에서 어떤 안건에 대해 표결을 할 때 찬성한다는 표시.
성적이나 등급을 나타내는 ‘수’, ‘우’, ‘미’, ‘양’, ‘가’의 다섯 단계 중에서 가장 낮은 등급.
어떤 행동이 가능하거나 허용됨.
['<구> 가를 꾸미다.\n<구> 가를 장식하다.\n<구> 가에 걸치다.\n<구> 가에 달다.\n<구> 가에 달라붙다.\n<구> 가에 세우다.\n<구> 가에 앉다.\n<문장> 공원의 중앙에는 잔디밭이 있고 가에는 울타리가 둘러쳐져 있었다.\n<문장> 민준이는 금방이라도 일어날 듯이 의자 가에 엉덩이만 살짝 걸치고 앉았다.\n<대화> 가: 차는 어디에 주차했어요?\n<대화> 나: 저기 운동장 가에 세워 뒀어요.\n\n', '<구> 강가.\n<구> 길가.\n<구> 냇가.\n<구> 문가.\n<구> 시냇가.\n<구> 우물가.\n<구> 창가.\n<구> 창문가.\n<구> 호숫가.\n\n', '<구> 가와 부.\n<구> 가를 밝히다.\n<문장> 곧 표결을 시작하오니 가인지 부인지 여부를 정해 주시기 바랍니다.\n<문장> 이 안건이 통과되는 것에 반대하는 사람이 적어 표가 가에 몰릴 것으로 예상된다.\n<대화> 가: 이 안건에 대해 가인지 부인지를 결정하셨습니까?\n<대화> 나: 아니요. 좀 더 신중하게 생각할 시간이 필요합니다.\n\n', '<구> 가가 나오다.\n<구> 가를 받다.\n<문장> 운동을 못 하는 지수는 체육 성적이 항상 가였다.\n<문장> 민준이는 공부를 잘해서 한 번도 가를 받은 적이 없다.\n<대화> 가: 네 성적표에 가도 있어?\n<대화> 나: 아니, 양은 두 개 있지만 가는 없어.\n\n', "<구> 관람 가.\n<구> 무이자 가.\n<구> 선택 가.\n<구> 입장 가.\n<구> 입주 가.\n<구> 할인 가.\n<문장> 우리 카페는 동물 입장 가입니다.\n<문장> 새로 지은 건물에는 '바로 입주 가'라고 써 있는 현수막이 걸려 있었다.\n<대화> 가: 열다섯 살인데 이 영화를 볼 수 있나요

In [92]:
tt = sorted(list(set(train['표제어'])))
len(tt)

42476

In [104]:
# test용 1000개
## 전체 데이터
import json
import random
thou = sorted(random.sample(tt, k=1000))
test = {}
with open('/content/test.jsonl', 'w', encoding='utf-8') as f:
    for i in thou:
        a = train[(train['표제어']==f'{i}')]
        b = '\n'.join(list(a['뜻풀이']))
        c = list(a['용례'])

        num1 = {
            "id": "alpaca_{idx}",
            "instruction": "다음 단어의 뜻을 알려주세요.",
            "input": f"{i}",
            "output": f"'{i}'의 뜻은 다음과 같습니다. {b}"
            }
        json.dump(num1, f, ensure_ascii=False)
        f.write('\n')

        num2 = {
            "id": "alpaca_{idx}",
            "instruction": f"'{i}'의 뜻을 알려주세요.",
            "input": "",
            "output": f"'{i}'의 뜻은 다음과 같습니다. {b}"
            }
        json.dump(num2, f, ensure_ascii=False)
        f.write('\n')

        num3 = {
            "id": "alpaca_{idx}",
            "instruction": "다음 단어의 예문을 알려주세요.",
            "input": f"{i}",
            "output": f"'{i}'의 예문은 다음과 같습니다. {c}"
            }
        json.dump(num3, f, ensure_ascii=False)
        f.write('\n')

        num4 = {
            "id": "alpaca_{idx}",
            "instruction": f"'{i}'의 예문을 알려주세요.",
            "input": f"",
            "output": f"'{i}'의 예문은 다음과 같습니다. {c}"
            }
        json.dump(num4, f, ensure_ascii=False)
        f.write('\n')


In [94]:
## 전체 데이터
import json

test = {}
with open('/content/train.json', 'w', encoding='utf-8') as f:
    for i in tt:
        a = train[(train['표제어']==f'{i}')]
        b = '\n'.join(list(a['뜻풀이']))
        c = list(a['용례'])

        num1 = {
            "id": "alpaca_{idx}",
            "instruction": "다음 단어의 뜻을 알려주세요.",
            "input": f"{i}",
            "output": f"'{i}'의 뜻은 다음과 같습니다. {b}"
            }
        json.dump(num1, f, ensure_ascii=False)
        f.write('\n')

        num2 = {
            "id": "alpaca_{idx}",
            "instruction": f"'{i}'의 뜻을 알려주세요.",
            "input": "",
            "output": f"'{i}'의 뜻은 다음과 같습니다. {b}"
            }
        json.dump(num2, f, ensure_ascii=False)
        f.write('\n')

        num3 = {
            "id": "alpaca_{idx}",
            "instruction": "다음 단어의 예문을 알려주세요.",
            "input": f"{i}",
            "output": f"'{i}'의 예문은 다음과 같습니다. {c}"
            }
        json.dump(num3, f, ensure_ascii=False)
        f.write('\n')

        num4 = {
            "id": "alpaca_{idx}",
            "instruction": f"'{i}'의 예문을 알려주세요.",
            "input": f"{i}",
            "output": f"'{i}'의 예문은 다음과 같습니다. {c}"
            }
        json.dump(num4, f, ensure_ascii=False)
        f.write('\n')

In [64]:
# with open('/content/train.json', 'w') as f:
#     json.dump(num1, f, indent="\t")

In [96]:
with open('/content/train.json') as f:
    data = [json.loads(line) for line in f]
data

[{'id': 'alpaca_{idx}',
  'instruction': '다음 단어의 뜻을 알려주세요.',
  'input': '가',
  'output': "'가'의 뜻은 다음과 같습니다. 어떤 장소나 물건의 둘레나 끝부분.\n‘주변’의 뜻을 나타내는 말.\n회의 등에서 어떤 안건에 대해 표결을 할 때 찬성한다는 표시.\n성적이나 등급을 나타내는 ‘수’, ‘우’, ‘미’, ‘양’, ‘가’의 다섯 단계 중에서 가장 낮은 등급.\n어떤 행동이 가능하거나 허용됨."},
 {'id': 'alpaca_{idx}',
  'instruction': "'가'의 뜻을 알려주세요.",
  'input': '',
  'output': "'가'의 뜻은 다음과 같습니다. 어떤 장소나 물건의 둘레나 끝부분.\n‘주변’의 뜻을 나타내는 말.\n회의 등에서 어떤 안건에 대해 표결을 할 때 찬성한다는 표시.\n성적이나 등급을 나타내는 ‘수’, ‘우’, ‘미’, ‘양’, ‘가’의 다섯 단계 중에서 가장 낮은 등급.\n어떤 행동이 가능하거나 허용됨."},
 {'id': 'alpaca_{idx}',
  'instruction': '다음 단어의 예문을 알려주세요.',
  'input': '가',
  'output': '\'가\'의 예문은 다음과 같습니다. [\'<구> 가를 꾸미다.\\n<구> 가를 장식하다.\\n<구> 가에 걸치다.\\n<구> 가에 달다.\\n<구> 가에 달라붙다.\\n<구> 가에 세우다.\\n<구> 가에 앉다.\\n<문장> 공원의 중앙에는 잔디밭이 있고 가에는 울타리가 둘러쳐져 있었다.\\n<문장> 민준이는 금방이라도 일어날 듯이 의자 가에 엉덩이만 살짝 걸치고 앉았다.\\n<대화> 가: 차는 어디에 주차했어요?\\n<대화> 나: 저기 운동장 가에 세워 뒀어요.\\n\\n\', \'<구> 강가.\\n<구> 길가.\\n<구> 냇가.\\n<구> 문가.\\n<구> 시냇가.\\n<구> 우물가.\\n<구> 창가.\\n<구> 창문가.\\n<구> 호숫가.\\n\\n\', \'<구> 가와 

### /